# Futures sum

Using dask futures API

In [1]:
import cudf


In [2]:
import dask
import distributed
import cudf
import os

In [3]:
print(f'Dask: {dask.__file__}')
print(f'Distributed: {distributed.__file__}')
print(f'cuDF: {cudf.__file__}')

Dask: /home/nfs/ashwint/miniconda3/envs/cudf_dev_92/lib/python3.7/site-packages/dask/__init__.py
Distributed: /home/nfs/ashwint/miniconda3/envs/cudf_dev_92/lib/python3.7/site-packages/distributed/__init__.py
cuDF: /home/nfs/ashwint/miniconda3/envs/cudf_dev_92/lib/python3.7/site-packages/cudf-0.10.0a0+800.g6407f50-py3.7-linux-x86_64.egg/cudf/__init__.py


In [4]:
base_env = {
    "UCX_RNDV_SCHEME": "put_zcopy",
    "UCX_MEMTYPE_CACHE": "n",
    "UCX_TLS": "rc,cuda_copy,cuda_ipc",
    "CUDA_VISIBLE_DEVICES": "0,1",
}
os.environ.update(base_env)

In [5]:
from dask.distributed import Client, wait
from dask_cuda import DGX

cluster = DGX(CUDA_VISIBLE_DEVICES=[0,1], 
              dashboard_address='10.33.227.165:8789')
client = Client(cluster)
# client = Client('ucx://10.33.225.165:42841')
client

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: ucx://10.33.225.165:51225
distributed.scheduler - INFO -   dashboard at:        10.33.225.165:8789
distributed.nanny - INFO -         Start Nanny at: 'ucx://10.33.225.165:36139'
distributed.nanny - INFO -         Start Nanny at: 'ucx://10.33.225.165:35355'
distributed.scheduler - INFO - Register ucx://10.33.225.165:60453
distributed.scheduler - INFO - Starting worker compute stream, ucx://10.33.225.165:60453
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register ucx://10.33.225.165:52940
distributed.scheduler - INFO - Starting worker compute stream, ucx://10.33.225.165:52940
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-65ab2836-c925-11e9-9289-d8c49764f624
distributed.core - INFO - Starting established connection


Client Scheduler: ucx://10.33.225.165:51225 Dashboard: http://10.33.225.165:8789/status,Cluster Workers: 2 Cores: 2 Memory: 270.53 GB


In [ ]:
from dask.distributed import wait
import operator


left = client.map(lambda x: cudf.Series([x]), range(100), workers=[0])
right = client.map(lambda x: cudf.Series([x]), range(100), workers=[1])
added = client.map(operator.add, left, right)
wait(added)